# Implementación de los protocolos

En este notebook vamos a crear una clase que contenga todos los protocolos. Será un conjunto ordenado congelado (sorted frozen set).

In [1]:
class SortedFrozenSet:
    
    def __init__(self, items=None):
        self._items = tuple(sorted(set(items) if (items is not None) else set()))
        
    #Protocolo Container
        
    def __contains__(self, item):
        return item in self._items
    
    #Protocolo Sized
    
    def __len__(self):
        return len(self._items)
    
    #Protocolo Iterable
    
    def __iter__(self):
        return iter(self._items)
    
    #Protocolo Sequence
    
    #Indexing and slicing
    
    def __getitem__(self, index):
        result = self._items[index]
        
        return (
            SortedFrozenSet(result)
            if isinstance(index, slice)
            else result
        )
    
    #String representation
    
    def __repr__(self):
        return "{type}({arg})".format(
            type=type(self).__name__,
            arg=(
                "[{}]".format(
                    ", ".join(
                        map(repr, self._items)
                    )
                )
                if self._items else ""
            )
        )
    
    # Equality operator
    
    def __eq__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return self._items == right_hand_side._items
    
    #Hashable protocol
    
    def __hash__(self):
        return hash(
            (type(self), self._items)
        )

En el inicializador, es necesario que se devuelva tupla porque es inmutable. Gracias a esto será posible crear el protocolo ***Hashable***.

Python contiene una librería llamada **abc (Abstract Base Classes)**, en el módulo **collections**. 

Esta librería nos ahorra mucho trabajo, ya que en caso de proporcionar una definición de los métodos:

- \__getitem__
- \__len__

La clase base nos proporcionará los siguientes métodos ya implementados:

- \__contains__
- \__iter__
- \__reversed__
- index
- count



En nuestro caso, nos interesa la colección **Sequence**:

In [2]:
from collections.abc import Sequence

Hacemos que nuestra clase herede de **Sequence**:

In [3]:
class SortedFrozenSet(Sequence):
    
    def __init__(self, items=None):
        self._items = tuple(sorted(set(items) if (items is not None) else set()))
        
    #Protocolo Container
        
    def __contains__(self, item):
        return item in self._items
    
    #Protocolo Sized
    
    def __len__(self):
        return len(self._items)
    
    #Protocolo Iterable
    
    def __iter__(self):
        return iter(self._items)
    
    #Protocolo Sequence (Todo lo de abajo)
    
    #Indexing and slicing
    
    def __getitem__(self, index):
        result = self._items[index]
        
        return (
            SortedFrozenSet(result)
            if isinstance(index, slice)
            else result
        )
    
    #String representation
    
    def __repr__(self):
        return "{type}({arg})".format(
            type=type(self).__name__,
            arg=(
                "[{}]".format(
                    ", ".join(
                        map(repr, self._items)
                    )
                )
                if self._items else ""
            )
        )
    
    # Equality operator
    
    def __eq__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return self._items == right_hand_side._items
    
    #Hashable protocol
    
    def __hash__(self):
        return hash(
            (type(self), self._items)
        )

## El protocolo Sequence extendido

Vamos a añadir los métodos ***add, mul y rmul***:

In [4]:
class SortedFrozenSet(Sequence):
    
    def __init__(self, items=None):
        self._items = tuple(sorted(set(items) if (items is not None) else set()))
        
    #Protocolo Container
        
    def __contains__(self, item):
        return item in self._items
    
    #Protocolo Sized
    
    def __len__(self):
        return len(self._items)
    
    #Protocolo Iterable
    
    def __iter__(self):
        return iter(self._items)
    
    #Protocolo Sequence (Todo lo de abajo)
    
    #Indexing and slicing
    
    def __getitem__(self, index):
        result = self._items[index]
        
        return (
            SortedFrozenSet(result)
            if isinstance(index, slice)
            else result
        )
    
    #String representation
    
    def __repr__(self):
        return "{type}({arg})".format(
            type=type(self).__name__,
            arg=(
                "[{}]".format(
                    ", ".join(
                        map(repr, self._items)
                    )
                )
                if self._items else ""
            )
        )
    
    # Equality operator
    
    def __eq__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return self._items == right_hand_side._items
    
    #Hashable protocol
    
    def __hash__(self):
        return hash(
            (type(self), self._items)
        )
    
    def __add__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return SortedFrozenSet(
            chain(self._items, right_hand_side._items)
        )
    
    def __mul__(self, right_hand_side):
        return self if right_hand_side > 0 else SortedFrozenSet
    
    def __rmul__(self, left_hand_side):
        return self * left_hand_side
    

La multiplicación en las listas funciona concatenando el mismo elemento a la lista tantas veces como indique el número por el que se multiplica. Por eso en **__mul__** se devuelve el propio elemento a menos que el número sea 0 o menor.

Es decir, **__mul__** devuelve lo que se quiere concatenar tantas veces como el número por el que se multiplica al objeto de la clase.

**__rmul__** se crea para cuando el objeto esté en la parte derecha del operador de multiplicación. 

Como podemos ver, dentro del método se pone en la parte izquierda.

## Refactorizando para mejorar el rendimiento

Para hacer la comprobación, vamos a utiliza la **secuencia de Recamán**.

In [5]:
from itertools import count

def recaman():
    
    seen = set()
    
    a = 0
    
    for n in count(1):
        yield a
        seen.add(a)
        c = a - n
        if c < 0 or c in seen:
            c = a + n
        a = c

Sacamos los primeros 50 números que devuelve la secuencia para ver que funciona:

In [6]:
from itertools import islice

list(islice(recaman(), 50))

[0,
 1,
 3,
 6,
 2,
 7,
 13,
 20,
 12,
 21,
 11,
 22,
 10,
 23,
 9,
 24,
 8,
 25,
 43,
 62,
 42,
 63,
 41,
 18,
 42,
 17,
 43,
 16,
 44,
 15,
 45,
 14,
 46,
 79,
 113,
 78,
 114,
 77,
 39,
 78,
 38,
 79,
 37,
 80,
 36,
 81,
 35,
 82,
 34,
 83]

Sacamos los primeros 1000 números:

In [7]:
s = SortedFrozenSet(r for r in islice(recaman(), 1000) if r < 1000)

In [8]:
len(s)

613

In [9]:
[s.count(i) for i in range(1000)]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [10]:
from timeit import timeit

timeit(setup='from __main__ import s', stmt='[s.count(i) for i in range(1000)]', number=200)

4.9389333

Si ahora usamos la funcion **bisect** de Python para refactorizar **__contains__, index** y **__count__**:

**bisect_left** devuelve la posición en de una lista ordenada en la que un elemento debe ser insertado para que la lista siga siendo ordenada. lo hace con complejidad logaritmica, por lo que es muy eficiente:

"*This function returns the position in the sorted list, where the number passed in argument can be placed so as to maintain the resultant list in sorted order. If the element is already present in the list, the left most position where element has to be inserted is returned.*"

Documentación: https://www.geeksforgeeks.org/bisect-algorithm-functions-in-python/#:~:text=%20%20%201%20bisect_left%20%28list%2C%20num%2C%20beg%2C,arguments%2C%20list%20which%20has%20to%20be...%20More%20

In [11]:
from bisect import bisect_left

In [12]:
class SortedFrozenSet(Sequence):
    
    def __init__(self, items=None):
        self._items = tuple(sorted(set(items) if (items is not None) else set()))
        
    #Protocolo Container
        
    def __contains__(self, item):
        try:
            self.index(item)
            return True
        except ValueError:
            return False
        
    #Protocolo Sized
    
    def __len__(self):
        return len(self._items)
    
    #Protocolo Iterable
    
    def __iter__(self):
        return iter(self._items)
    
    #Protocolo Sequence (Todo lo de abajo)
    
    #Indexing and slicing
    
    def __getitem__(self, index):
        result = self._items[index]
        
        return (
            SortedFrozenSet(result)
            if isinstance(index, slice)
            else result
        )
    
    #String representation
    
    def __repr__(self):
        return "{type}({arg})".format(
            type=type(self).__name__,
            arg=(
                "[{}]".format(
                    ", ".join(
                        map(repr, self._items)
                    )
                )
                if self._items else ""
            )
        )
    
    # Equality operator
    
    def __eq__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return self._items == right_hand_side._items
    
    #Hashable protocol
    
    def __hash__(self):
        return hash(
            (type(self), self._items)
        )
    
    def __add__(self, right_hand_side):
        if not isinstance(right_hand_side, type(self)):
            return NotImplemented
        return SortedFrozenSet(
            chain(self._items, right_hand_side._items)
        )
    
    def __mul__(self, right_hand_side):
        return self if right_hand_side > 0 else SortedFrozenSet
    
    def __rmul__(self, left_hand_side):
        return self * left_hand_side
    
    def count(self, item):
        return int(item in self)
    
    def index(self, item):
        index = bisect_left(self._items, item)
        
        # Si el indice donde debe ser insertado está dentro de los limites de la tupla
        # y 
        # si el elemento de ese indice es equivalente al que estamos buscando
        
        if (index != len(self._items)) and self._items[index] == item:
            return index
        raise ValueError(f"{item!r} not found.")

In [13]:
s = SortedFrozenSet(r for r in islice(recaman(), 1000) if r < 1000)

In [14]:
from timeit import timeit

timeit(setup='from __main__ import s', stmt='[s.count(i) for i in range(1000)]', number=200)

0.21595310000000012

Como podemos ver, la diferencia de tiempo es brutal.